In [1]:
import os
os.chdir('C:/Users/theodore.wong/cif/cities-cif')
import ee, geemap, json
import geopandas as gpd
import pandas as pd
import shapely
import rioxarray
#ee.Authenticate()
ee.Initialize()
from city_metrix.layers import Layer, OpenStreetMap, OpenStreetMapClass, WorldPop
from city_metrix.layers.layer import get_utm_zone_epsg

import warnings
warnings.filterwarnings('ignore')


FILEPATH = 'C:/Users/theodore.wong/danida_africa'

Could not find GEE credentials file, so prompting authentication.


In [2]:
boundary_filenames = os.listdir('{0}/boundaries'.format(FILEPATH))
boundary_filenames

['boundary-ETH-Dire_Dawa-ADM3.geojson',
 'boundary-ETH-Dire_Dawa-ADM3union.geojson',
 'boundary-KEN-Nairobi-ADM3.geojson',
 'boundary-KEN-Nairobi-ADM3union.geojson',
 'boundary-RWA-Kigali-ADM3.geojson',
 'boundary-RWA-Kigali-ADM4union.geojson',
 'boundary-RWA-Musanze-ADM2union.geojson',
 'boundary-RWA-Musanze-ADM3.geojson',
 'boundary-ZAF-CityOfJohannesburg-ADM3union.geojson',
 'boundary-ZAF-CityOfJohannesburg-ADM4.geojson',
 'boundary-ZAF-NelsonMandelaBay-ADM3union.geojson',
 'boundary-ZAF-NelsonMandelaBay-ADM4.geojson',
 'z_boundary-MEX-Guadalajara-metroarea-cities.geojson']

In [3]:
def buffer_gdf(gdf, buffer_distance_meters):
    source_crs = gdf.crs
    target_epsg = get_utm_zone_epsg(gdf.to_crs('EPSG:4326').total_bounds)
    gdf = gdf.to_crs(target_epsg)
    return gpd.GeoDataFrame({'geometry': gdf.buffer(buffer_distance_meters).to_crs('EPSG:4326')})

In [327]:
class IsochroneVector(Layer):
    def __init__(self, gdf, min_threshold=0, **kwargs):
        super().__init__(**kwargs)
        self.spatial_resolution = 100
        self.min_threshold = min_threshold
        self.gdf = gpd.GeoDataFrame({'is_accessible': [1] * len(gdf), 'geometry': gdf.to_crs('EPSG:4326')['geometry']}).set_crs('EPSG:4326').set_geometry('geometry')
        
    def get_data(self, bbox):
        return self.gdf.clip(shapely.box(*bbox))

In [476]:
big_box = buffer_gdf(zones, 500).total_bounds
big_box

array([ 25.1868026 , -34.05526847,  25.87355393, -33.5485737 ])

In [484]:
class IsochroneVectorFromGeoJSON(Layer):
    def __init__(self, filename, min_threshold=0, **kwargs):
        super().__init__(**kwargs)
        self.spatial_resolution = 100
        self.min_threshold = min_threshold
        gdf = gpd.read_file(filename)
        self.gdf = gpd.GeoDataFrame({'is_accessible': [1] * len(gdf), 'geometry': gdf.to_crs('EPSG:4326')['geometry']}).to_crs('EPSG:4326').set_crs('EPSG:4326').set_geometry('geometry')
        self.filename = filename
        
    def get_data(self, bbox):
        return self.gdf.clip(shapely.box(*bbox))

In [474]:
for fname in boundary_filenames:
    if fname.split('-')[3].split('.')[0][-1] != 'n':
        cityname = '{0} {1}'.format(fname.split('-')[1], fname.split('-')[2])
        print(cityname)
        with open('{0}/boundaries/{1}'.format(FILEPATH, fname), 'r') as ifile:
            json_obj = json.loads(ifile.read())
        feature = geemap.geojson_to_ee(json_obj)
        zones = geemap.ee_to_gdf(feature)
        schools = OpenStreetMap(OpenStreetMapClass.SCHOOLS).get_data(buffer_gdf(zones, 500).total_bounds)
        if len(schools):
            schools_buffered = buffer_gdf(schools, 500)
        else:
            schools_buffered = schools
        with open('{0}/schools_prebuf_{1}.geojson'.format(FILEPATH, cityname.replace(' ', '-')), 'w') as ofile:
            ofile.write(schools_buffered.to_json())

ETH Dire_Dawa
KEN Nairobi
RWA Kigali
RWA Musanze
ZAF CityOfJohannesburg
ZAF NelsonMandelaBay
MEX Guadalajara


In [470]:
def accessible_population_gdf(access_features_layer, popraster_layer, zones):
    if len(access_features_layer.gdf):
        result_series = popraster_layer.mask(access_features_layer).groupby(zones).sum()
    else:
        result_series = pd.Series([0] * len(zones))
    return result_series

In [519]:
def get_result_gdf(population_layer, accesszone_layer, zones):
    try:
        access_pop = accessible_population_gdf(accesszone_layer, population_layer, zones)
        total_pop = population_layer.groupby(zones).sum()
        zones['total_population'] = total_pop
        zones['accessible_population'] = access_pop
        zones['accessible_fraction'] = access_pop / total_pop
        return zones
    except:
        print('*** Trying dropping a district ***')
        idx = 0
        while idx < len(zones.index):
            try:
                mostrows_gdf = zones.drop(zones.index[idx]).reset_index()
                access_pop = accessible_population_gdf(accesszone_layer, population_layer, mostrows_gdf)
                total_pop = population_layer.groupby(mostrows_gdf).sum()
                mostrows_gdf['total_population'] = total_pop
                mostrows_gdf['accessible_population'] = access_pop
                mostrows_gdf['accessible_fraction'] = access_pop / total_pop
                dropped_gdf = zones.loc[[zones.index[idx]]]
                dropped_access_pop = accessible_population_gdf(accesszone_layer, population_layer, dropped_gdf)
                dropped_total_pop = population_layer.groupby(dropped_gdf).sum()
                dropped_gdf['total_population'] = dropped_total_pop
                dropped_gdf['accessible_population'] = dropped_access_pop
                dropped_gdf['accessible_fraction'] = dropped_access_pop / dropped_total_pop
                print('*** FOUND RESULT WITH IDX', idx)
                return pd.concat([mostrows_gdf, dropped_gdf]).reset_index()
            except:
                idx += 1
        zones['total_population'] = None
        zones['accessible_population'] = None
        zones['accessible_fraction'] = None
        return zones
            

In [522]:
population_layer = WorldPop()

for fname in boundary_filenames:
    if fname.split('-')[3].split('.')[0][-1] != 'n':
        cityname = '{0} {1}'.format(fname.split('-')[1], fname.split('-')[2])
        print(cityname)
        with open('{0}/boundaries/{1}'.format(FILEPATH, fname), 'r') as ifile:
            json_obj = json.loads(ifile.read())
        feature = geemap.geojson_to_ee(json_obj)
        zones = geemap.ee_to_gdf(feature)
        schools_layer = IsochroneVectorFromGeoJSON('{0}/schools_prebuf_{1}.geojson'.format(FILEPATH, cityname.replace(' ', '-')))
        result = get_result_gdf(population_layer, schools_layer, zones)
        with open('{0}/schools_accessfraction_{1}.geojson'.format(FILEPATH, cityname.replace(' ', '-')), 'w') as ofile:
            ofile.write(zones.to_json())

ETH Dire_Dawa
Input covers too much area, splitting into 2 tiles


C:\Users\theodore.wong\AppData\Local\anaconda3\envs\cif\lib\site-packages\xee\ext.py:685: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


Extracting layer world pop from Google Earth Engine for bbox [41.63315988  9.44538064 42.13315988  9.79335473]:
[########################################] | 100% Completed | 1.81 ss


C:\Users\theodore.wong\AppData\Local\anaconda3\envs\cif\lib\site-packages\xee\ext.py:685: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


Extracting layer world pop from Google Earth Engine for bbox [42.13315988  9.57078713 42.42197622  9.84160302]:
[########################################] | 100% Completed | 925.17 ms


...

C:\Users\theodore.wong\AppData\Local\anaconda3\envs\cif\lib\site-packages\xee\ext.py:685: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


Extracting layer world pop from Google Earth Engine for bbox [-103.1884244    20.32698925 -102.869427     20.7961688 ]:
[########################################] | 100% Completed | 1.16 sms


C:\Users\theodore.wong\AppData\Local\anaconda3\envs\cif\lib\site-packages\xee\ext.py:685: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


Extracting layer world pop from Google Earth Engine for bbox [-103.6552783   20.8257581 -103.3252144   20.9982375]:
[########################################] | 100% Completed | 584.79 ms


C:\Users\theodore.wong\cif\cities-cif\city_metrix\layers\layer.py:151: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return aggregated.reset_index()
